In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import copy
from os import path

from pprint import pprint

from versuchung.search import *

import sys
import latency

from versuchung.tex import DatarefDict

raw = {}

hosts = ['armorix', 'zynqmp', 'rpi4']
variants = ['pac-pl', 'syscall', 'kpacd']
backends = ['nop', 'siphash', 'qarma']

datapath = "data/"
for host in hosts:
    raw[host] = {}
    for backend in backends:
        raw[host][backend] = {}
        for var in variants:
            if var == 'pac-pl' and not (host == 'zynqmp' and (backend == 'qarma' or backend == 'nop')):
                continue
            print(host, backend, var)
            exp = search_experiment(latency.Latency, datapath, {'host': host, 'variant': var, 'backend': backend})
            print(exp.path)
            raw[host][backend][var] = dict(np.load(exp.o.samples.path))['arr_0']

dfs = {k: pd.DataFrame(v) for k,v in raw.items()}

pprint(dfs)

armorix nop syscall
/srv/scratch/ill.ostapyshyn/proj/kpac/rtas-pac/data/latency/data/Latency-eee3c72a254bd5623221b51aed6813d6
armorix nop kpacd
/srv/scratch/ill.ostapyshyn/proj/kpac/rtas-pac/data/latency/data/Latency-6c1ed394fa1f44578f1ef8468b95dc73
armorix siphash syscall
/srv/scratch/ill.ostapyshyn/proj/kpac/rtas-pac/data/latency/data/Latency-bbf2c0b088b4bc2d0590f247806bc924
armorix siphash kpacd
/srv/scratch/ill.ostapyshyn/proj/kpac/rtas-pac/data/latency/data/Latency-7f803248fb359278d73452d0cc3d7b5e
armorix qarma syscall
/srv/scratch/ill.ostapyshyn/proj/kpac/rtas-pac/data/latency/data/Latency-0430d0d8a40c51017ff5bbc483d7b1d7
armorix qarma kpacd
/srv/scratch/ill.ostapyshyn/proj/kpac/rtas-pac/data/latency/data/Latency-5e33c77ea5a355aed790d6082dd02f1f
zynqmp nop pac-pl
/srv/scratch/ill.ostapyshyn/proj/kpac/rtas-pac/data/latency/data/Latency-85140f41cbf7936bda8fe2b27c88b15c
zynqmp nop syscall
/srv/scratch/ill.ostapyshyn/proj/kpac/rtas-pac/data/latency/data/Latency-f5bc6d33f04ee4bc0b9ddb

In [5]:
means = {k: v.map(np.mean).T for k,v in dfs.items()}
#p50s = {k: v.map(lambda x: np.percentile(x, q=50)).T for k,v in dfs.items()}
p99s = {k: v.map(lambda x: np.percentile(x, q=99)).T for k,v in dfs.items()}
#p999s = {k: v.map(lambda x: np.percentile(x, q=99.9)).T for k,v in dfs.items()}

#stds = {k: v.map(lambda x: np.std(x)/np.mean(x)) for k,v in dfs.items()}

display(means)
#display(p50s)
display(p99s)
#display(p999s)

#display(stds)

{'armorix':              syscall       kpacd
 nop       388.714135   440.19870
 siphash   434.274295   453.28932
 qarma    3864.984505  3900.12632,
 'zynqmp':              pac-pl       syscall        kpacd
 nop      635.696597   2000.756839   190.821823
 siphash         NaN   2101.956101   318.003957
 qarma    646.422423  11530.450251  8256.013216,
 'rpi4':              syscall        kpacd
 nop      2023.574244   360.125496
 siphash  2128.489164   449.535060
 qarma    6739.242172  5315.718693}

{'armorix':          syscall   kpacd
 nop        389.0   476.0
 siphash    434.0   488.0
 qarma     3903.0  3947.0,
 'zynqmp':          pac-pl  syscall   kpacd
 nop       643.0   2020.0   217.0
 siphash     NaN   2122.0   339.0
 qarma     650.0  11608.0  8231.0,
 'rpi4':          syscall   kpacd
 nop       2095.0   420.0
 siphash   2170.0   508.0
 qarma     6779.0  5332.0}

In [6]:
for host in hosts:
    dref_file = f"{host}.tex"
    try:
        os.unlink(dref_file)
    except OSError:
        pass

    dref_out = DatarefDict(dref_file)
    dref_out.pandas(means[host], prefix='mean')
    dref_out.pandas(p99s[host], prefix='p99')
    #dref_out.pandas(p999s[host], prefix='p999')
    dref_out.flush()